In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import argparse

import matplotlib.pyplot as plt
import matplotlib.colors as col
import seaborn as sns
import plotly.express as px
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel, permutation_test
from statsmodels.stats import multitest

from tfsemb_class import tsne, save_pickle, add_speech
from tfsplt_encoding import get_cmap_smap, get_sid, aggregate_data, organize_data
from tfsplt_utils import read_folder, load_pickle, get_con_color, get_cat_color
from tfsplt_brainmap import get_sigelecs, Colorbar, make_brainmap
from tfsplt_brainmap_cat import make_brainmap_cat

plt.style.use('/scratch/gpfs/ln1144/247-plotting/scripts/paper.mlpstyle')
# plt.style.use('../data/plotting/paper-prob-improb/paper.mlpstyle')
ls = '-'
lw = 1

## Pred Lag Compare

In [ ]:
def aggregate_data_folds(args):
    """Aggregate encoding data

    Args:
        args (namespace): commandline arguments

    Returns:
        df (DataFrame): df with all encoding results
    """
    print("Aggregating data")

    def read_file(fname):
        files = glob.glob(fname)
        assert (
            len(files) > 0
        ), f"No results found under {fname}"  # check files exist under format

        for resultfn in files:
            elec = os.path.basename(resultfn).replace(".csv", "")[:-10]
            # Skip electrodes if they're not part of the sig list
            if len(args.sigelecs) and elec not in args.sigelecs[(load_sid, key)]:
                continue
            df = pd.read_csv(resultfn, header=None)
            df = df.iloc[[10]]
            df.insert(0, "sid", load_sid)
            df.insert(0, "key", key)
            df.insert(0, "electrode", elec)
            df.insert(0, "label", label)
            data.append(df)

    data = []

    if len(args.labels) / len(args.unique_labels) == len(args.sid):
        for fmt, label in zip(args.formats, args.labels):
            load_sid = get_sid(fmt, args)
            for key in args.keys:
                fname = fmt % key
                read_file(fname)
    else:
        for load_sid in args.sid:
            for fmt, label in zip(args.formats, args.labels):
                for key in args.keys:
                    fname = fmt % (load_sid, key)
                    read_file(fname)

    if not len(data):
        print("No data found")
        exit(1)
    df = pd.concat(data)
    return df

In [ ]:
class Args(argparse.Namespace):
  sid = [625, 676, 7170, 798] # subjects
  project = "tfs"
  formats = [ # encoding folder
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-0/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-4/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-decoder-nots-lag2k-25-all-noearlypca/*/*%s.csv",
    # "../data/encoding/tfs/20231108-whisper-acoustic/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all/*/*%s.csv",
    "../data/encoding/tfs/20231118-decoder-ts-scare/kw-tfs-full-%s-whisper-tiny.en-decoder-nots-lag10k-25-all-noearlypca/*/*%s.csv",
  ]
  labels = [
    "encoder"
  ]
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-de-last-0.01-%(key)s.csv"]
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(-10000,10001,25) # encoding lags
  lags_show = np.arange(-10000,10001,25) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes

# for item in args.sigelecs: # align prod/comp elecs
#     comp_set = set(args.sigelecs[(item[0], "comp")])
#     prod_set = set(args.sigelecs[(item[0], "prod")])
#     args.sigelecs[item] = comp_set.union(prod_set)

df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary
df.sort_index(level=[3,1,2], ascending=True, inplace=True)
df = df.droplevel("label")
df["max"] = df.max(axis=1)

In [ ]:
class Args(argparse.Namespace):
  sid = [625, 676, 7170, 798] # subjects
  project = "tfs"
  formats = [ # encoding folder
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-0-pred-lag/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-pred-lag/*/*%s.csv",
    "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-decoder-nots-lag2k-25-all-noearlypca-pred-lag/*/*%s.csv",
  ]
  labels = [
    "pred-lag"
  ]
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-de-last-0.01-%(key)s.csv"]
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(0,1,1) # encoding lags
  lags_show = np.arange(0,1,1) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes

df2 = aggregate_data(args) # aggregate data
df2 = organize_data(args, df2) # trim data if necessary
df2["max"] = df2.max(axis=1)

df2.sort_index(level=[3,1,2], ascending=True, inplace=True)
df2 = df2.droplevel("label")
df2.loc[:,"original"] = df["max"]

In [ ]:
df2_comp = df2.loc[df2.index.get_level_values("key") == "comp"]
df2_prod = df2.loc[df2.index.get_level_values("key") == "prod"]

In [ ]:
fig, ax = plt.subplots()
plt.scatter(df2_comp["original"], df2_comp["max"],s=3)
ax.plot([0, 1], [0, 1], transform=ax.transAxes)
ax.set_ylim(0,0.4)
ax.set_xlim(0,0.4)
plt.savefig(f"../de-comp.svg")

fig, ax = plt.subplots()
plt.scatter(df2_prod["original"], df2_prod["max"],s=3)
ax.plot([0, 1], [0, 1], transform=ax.transAxes)
ax.set_ylim(0,0.4)
ax.set_xlim(0,0.4)
plt.savefig(f"../de-prod.svg")

## Binsize & Samplesize

In [ ]:
class Args(argparse.Namespace):
  sid = [625, 676, 7170, 798] # subjects
  project = "tfs"
  formats = [ # encoding folder
    "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all/*/*_%s.csv",
    "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all-100/*/*_%s.csv",
    "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all-50/*/*_%s.csv",
    # "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-all-noearlypca/*/*_%s.csv",
    # "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-all-noearlypca-100/*/*_%s.csv",
    # "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-all-noearlypca-50/*/*_%s.csv",
  ]
  labels = [
    "win-200",
    "win-100",
    "win-050"
  ]
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-en-last-0.01-%(key)s.csv"]
  # sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-de-best-0.01-%(key)s.csv"]
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(-10000,10001,25) # encoding lags
  lags_show = np.arange(-2000,2001,25) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes
colors = get_cat_color()
styles = ["-", "-", "-", "-"]
cmap = {}  # line color map
smap = {}  # line style map
for label, color in zip(args.unique_labels, colors):
  for key, style in zip(args.unique_keys, styles):
    cmap[(label, key)] = color
    smap[(label, key)] = style
args.cmap = cmap
args.smap = smap

df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary

In [ ]:
class Args(argparse.Namespace):
  sid = [625, 676, 7170, 798] # subjects
  project = "tfs"
  formats = [ # encoding folder
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-all/*/*_%s.csv",
    # "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-half/*/*_%s.csv",
    # "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-quarter/*/*_%s.csv",
    # "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-encoder-lag10k-25-tenth/*/*_%s.csv",
    "../data/encoding/tfs/20231004-whisper-binsize/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-all-noearlypca/*/*_%s.csv",
    "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-half/*/*_%s.csv",
    "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-quarter/*/*_%s.csv",
    "../data/encoding/tfs/20231018-whisper-sample-size/kw-tfs-full-%s-whisper-tiny.en-decoder-lag10k-25-tenth/*/*_%s.csv",
  ]
  labels = [
    "all",
    "half",
    "quarter",
    "tenth"
  ]
  # sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-en-last-0.01-%(key)s.csv"]
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-de-best-0.01-%(key)s.csv"]
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(-10000,10001,25) # encoding lags
  lags_show = np.arange(-2000,2001,25) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes
colors = get_cat_color()
styles = ["-", "-", "-", "-"]
cmap = {}  # line color map
smap = {}  # line style map
for label, color in zip(args.unique_labels, colors):
  for key, style in zip(args.unique_keys, styles):
    cmap[(label, key)] = color
    smap[(label, key)] = style
args.cmap = cmap
args.smap = smap

df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary

## Encoding Plot

In [ ]:
for plot, subdf in df.groupby("key", axis=0):
    fig, ax = plt.subplots()
    for line, subsubdf in subdf.groupby("label", axis=0):
        vals = subsubdf.mean(axis=0)
        err = subsubdf.sem(axis=0)
        map_key = (line, plot)
        ax.fill_between(
            args.lags_show,
            vals - err,
            vals + err,
            alpha=0.2,
            color=args.cmap[map_key],
        )
        ax.plot(
            args.lags_show,
            vals,
            label=f"{line} ({len(subsubdf)})",
            color=args.cmap[map_key],
            ls=args.smap[map_key],
        )
        ax.axhline(0, ls="dashed", alpha=0.3, c="k")
        ax.axvline(0, ls="dashed", alpha=0.3, c="k")
        ax.set_title(f"{plot} global average")
        ax.legend(loc="upper right", frameon=False)
        ax.set(xlabel="Lag (s)", ylabel="Correlation (r)")
        plt.savefig(f"../de-samplesize-{plot}.svg")

## Scatter Plot

In [ ]:
def get_part_df(label):  # get partial df
    idx = pd.IndexSlice
    part_df = df.loc[idx[label, :, :, :], :].copy()
    part_df.index = part_df.index.droplevel("label")
    part_df_idx = part_df.index.get_level_values("electrode").tolist()
    return part_df, part_df_idx

df1, df1_idx = get_part_df("all")
df2, df2_idx = get_part_df("quarter")
df1["max"] = df1.max(axis=1)
df2["max"] = df2.max(axis=1)
df1.sort_index(level=[2,0,1], ascending=True, inplace=True)
df2.sort_index(level=[2,0,1], ascending=True, inplace=True)
df1["max-quarter"] = df2["max"]
df1_comp = df1.loc[df1.index.get_level_values("key") == "comp"]
df1_prod = df1.loc[df1.index.get_level_values("key") == "prod"]

In [ ]:
fig, ax = plt.subplots()
plt.scatter(df1_comp["max"], df1_comp["max-quarter"],s=3)
ax.plot([0, 1], [0, 1], transform=ax.transAxes)
ax.set_ylim(0,0.4)
ax.set_xlim(0,0.4)
plt.savefig(f"../quarter-comp.svg")

fig, ax = plt.subplots()
plt.scatter(df1_prod["max"], df1_prod["max-quarter"],s=3)
ax.plot([0, 1], [0, 1], transform=ax.transAxes)
ax.set_ylim(0,0.4)
ax.set_xlim(0,0.4)
plt.savefig(f"../quarter-prod.svg")

## Noise Ceiling

In [ ]:
def aggregate_data_ceiling(args, row_num, label_name):
    print("Aggregating data")

    def read_file(fname, row_num, label):
        files = glob.glob(fname)
        assert (
            len(files) > 0
        ), f"No results found under {fname}"  # check files exist under format

        for resultfn in files:
            elec = os.path.basename(resultfn).replace(".csv", "")[:-5]
            # Skip electrodes if they're not part of the sig list
            if len(args.sigelecs) and elec not in args.sigelecs[(load_sid, key)]:
                continue
            df = pd.read_csv(resultfn, header=None)
            df = df.iloc[[row_num]]
            df.insert(0, "sid", load_sid)
            df.insert(0, "key", key)
            df.insert(0, "electrode", elec)
            df.insert(0, "label", label)
            data.append(df)

    data = []

    if len(args.labels) / len(args.unique_labels) == len(args.sid):
        for fmt, label in zip(args.formats, args.labels):
            load_sid = get_sid(fmt, args)
            for key in args.keys:
                fname = fmt % key
                read_file(fname, row_num, label_name)
    else:
        for load_sid in args.sid:
            for fmt, label in zip(args.formats, args.labels):
                for key in args.keys:
                    fname = fmt % (load_sid, key)
                    read_file(fname, row_num, label_name)

    if not len(data):
        print("No data found")
        exit(1)
    df = pd.concat(data)
    return df

In [ ]:
class Args(argparse.Namespace):
  sid = [625, 676, 7170, 798] # subjects
  project = "tfs"
  formats = [ # encoding folder
    "../data/encoding/tfs/20231221-noise/kw-tfs-full-%s-whisper-tiny.en-decoder-nots-lag10k-25-all-noise-ceiling/*/*%s.csv",
  ]
  labels = [
    "noise"
  ]
  sig_elec_file = []
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(-10000,10001,25) # encoding lags
  lags_show = np.arange(-10000,10001,25) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes

# df1 = aggregate_data_ceiling(args, 0 ,"min") # aggregate data
# df2 = aggregate_data_ceiling(args, 1 ,"25") # aggregate data
# df3 = aggregate_data_ceiling(args, 2 ,"50") # aggregate data
# df4 = aggregate_data_ceiling(args, 3 ,"75") # aggregate data
# df5 = aggregate_data_ceiling(args, 4 ,"max") # aggregate data
# df6 = aggregate_data_ceiling(args, 5 ,"mean") # aggregate data
# df7 = aggregate_data_ceiling(args, 6 ,"std") # aggregate data
# df8 = aggregate_data_ceiling(args, 7 ,"sem") # aggregate data

save_path = "../data/plotting/paper-whisper/data/noise_ceiling/"
# df1.to_csv(os.path.join(save_path, "min.csv"),index=False)
# df2.to_csv(os.path.join(save_path, "25.csv"),index=False)
# df3.to_csv(os.path.join(save_path, "50.csv"),index=False)
# df4.to_csv(os.path.join(save_path, "75.csv"),index=False)
# df5.to_csv(os.path.join(save_path, "max.csv"),index=False)
# df6.to_csv(os.path.join(save_path, "mean.csv"),index=False)
# df7.to_csv(os.path.join(save_path, "std.csv"),index=False)
# df8.to_csv(os.path.join(save_path, "sem.csv"),index=False)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,20))

modes = ["min","25","50","75","max","mean"]
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
]

for mode, color in zip(modes, colors):
    df = pd.read_csv(os.path.join(save_path, f"{mode}.csv"))
    df = organize_data(args, df) # trim data if necessary
    means = df.mean(axis=0)
    sems = df.sem(axis=0)

    ax.fill_between(
        args.lags_show,
        means - sems,
        means + sems,
        alpha=0.2,
        color=color,
    )
    ax.plot(
        args.lags_show,
        means,
        label=mode,
        color=color,
        ls="-",
    )
    ax.legend(loc="upper right", frameon=False)
plt.savefig("noise.jpeg")

## Last Minute Brainmaps

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625,676,7170,798] # subjects
  formats = [ # encoding folder
    "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-0-pred-lag/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-encoder-lag2k-25-all-pred-lag/*/*%s.csv",
    # "../data/encoding/tfs/20231121-pick-lag/kw-tfs-full-%s-whisper-tiny.en-decoder-nots-lag2k-25-all-noearlypca-pred-lag/*/*%s.csv",
  ]
  labels = [
    "pred-lag"
  ]
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-en-first-0.01-%(key)s.csv"]
  keys = ["comp","prod"] # comprehension and/or production
  layers = np.arange(0,25)
  lags_plot = np.arange(0,1,1) # encoding lags
  lags_show = np.arange(0,1,1) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../tfs_%s.svg"
  final = True
  final2 = False
  shiny = False

args = Args()
args.color_split = [Colorbar(bar_min=0.04, bar_max=0.4)]
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes

df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary
df["effect"] = df.max(axis=1)
df.reset_index(drop=False, inplace=True)

fig = make_brainmap(args, df.loc[df.key=="comp",("electrode","effect")], args.outfile % "comp") # plot png
fig = make_brainmap(args, df.loc[df.key=="prod",("electrode","effect")], args.outfile % "prod") # plot png


In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625,676,7170,798] # subjects
  keys = ["comp","prod"] # comprehension and/or production
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../tfs_%s.svg"
  final = True
  final2 = False
  shiny = False

args = Args()

pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "#ffffff"],[1, "#ea0071"]],bar_min=0,bar_max=0.05)
neg_bar = Colorbar(title="Δ corr neg",colorscale=[[0, "#0000ff"],[1, "#ffffff"]],bar_min=-0.05,bar_max=0)
args.color_split = [neg_bar,0,pos_bar]

df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/text_only_vs_text_audio_de_text_only_vs_text_audio_comp.csv")
df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
fig = make_brainmap(args, df, args.outfile % "comp") # plot png
df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/text_only_vs_text_audio_de_text_only_vs_text_audio_prod.csv")
df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
fig = make_brainmap(args, df, args.outfile % "prod") # plot png

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625,676,7170,798] # subjects
  keys = ["comp","prod"] # comprehension and/or production
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../tfs_%s.svg"
  final = True
  final2 = False
  shiny = False

args = Args()
# Green & Purple whisper
# pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "#00fa9a"],[1, "#006400"]],bar_min=0.01,bar_max=0.15)
pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "#ee82ee"],[1, "#800080"]],bar_min=0.01,bar_max=0.15)
neg_bar = Colorbar(title="Δ corr neg",colorscale=[[0, "#006400"],[1, "#00fa9a"]],bar_min=-0.15,bar_max=-0.01)
args.color_split = [neg_bar,0,pos_bar]

df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/prod_comp_contrast/prod_comp_encoder.csv")
df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
fig = make_brainmap(args, df, args.outfile % "en") # plot png
df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/prod_comp_contrast/prod_comp_decoder.csv")
df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
fig = make_brainmap(args, df, args.outfile % "de") # plot png

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625,676,7170,798] # subjects
  keys = ["comp","prod"] # comprehension and/or production
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../tfs_%s.svg"
  final = True
  final2 = False
  shiny = False

args = Args()
# args.color_split = [Colorbar(title="Corr",colorscale=[[0, "#ffffff"],[1, "#ff0000"]],bar_min=0,bar_max=1)]
args.color_split = [Colorbar(title="Corr",colorscale=[[0, "#ffffff"],[1, "#0000ff"]],bar_min=0,bar_max=1)]

# df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/pc_flip/pc_flip_encoder.csv")
# df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
# fig = make_brainmap(args, df, args.outfile % "en") # plot png
df = pd.read_csv("/scratch/gpfs/ln1144/247-plotting/paper/materials/pc_flip/pc_flip_decoder.csv")
df["electrode"] = df.sid.astype(str) + "_" + df.elec_1
fig = make_brainmap(args, df, args.outfile % "de") # plot png